In [1]:
from torch import nn
import torch.nn.functional as F
import numpy as np
import torch
import random
import matplotlib.pyplot as plt
import re
from tqdm import tqdm

In [5]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
context_length = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 300
n_embd = 384
n_layers = 6
dropout = 0.2
n_heads = 6

rope_embeddings = True



In [58]:
def remove_non_punjabi_chars(text):
    punjabi_chars = r"[\u0A01-\u0A7F\u0A80-\u0A8F,।0-9? \n‘]"  # Gurmukhi range
    english_chars = r"[a-zA-Z]"  # English alphabet range
    text =  re.sub(r"[^" + punjabi_chars +"|"+ english_chars + "]+", " ", text) 
    text = re.sub(r" +", " ", text)
    text = re.sub(r"\n+", "\n", text)
    return text

# reading the punjabi corpus

with open('data/pa.txt') as file:
    punj_data = file.read()

# Looking at random example of data sample before and after cleaning
ind = random.randint(0, len(data)-500)

print(f'Data before cleaning: {punj_data[ind:ind+500]}\n')
print(f'Data after  cleaning: {remove_non_punjabi_chars(punj_data[ind:ind+500])}\n')



Data before cleaning: ੈਟਰੋਲ ਤੇ ਡੀਜ਼ਲ ਦੇ ਭਾਅ 'ਚ ਕੀਤੇ ਗਏ ਅਥਾਹ ਵਾਧੇ ਅਤੇ ਪੰਜਾਬ ਸਰਕਾਰ ਵਲੋਂ ਚੋਣਾ ਦੌਰਾਨ ਗਰੀਬਾਂ ਤੇ ਮਜਦੂਰਾਂ ਨੂੰ 5-5 ...
ਅੰਮਿ੍ਤਸਰ, 15 ਅਕਤੂਬਰ (ਹਰਮਿੰਦਰ ਸਿੰਘ)-ਪੰਜਾਬ ਦੇ ਕੈਬਨਿਟ ਮੰਤਰੀ ਨਵਜੋਤ ਸਿੰਘ ਸਿੱਧੂ ਦੇ ਿਖ਼ਲਾਫ਼ ਭਾਜਪਾ ਦੀ ਜ਼ਿਲ੍ਹਾ ਇਕਾਈ ਵਲੋਂ ਸਥਾਨਕ ਹਾਥੀ ਗੇਟ ਦੇ ਬਾਹਰਵਾਰ ਜ਼ੋਰਦਾਰ ਮੁਜ਼ਾਹਰਾ ਕੀਤਾ ਗਿਆ ਤੇ ਉਸ ਦਾ ਪੁਤਲਾ ਸਾੜਿਆ ਗਿਆ _ ਇਸ ਮੁਜ਼ਾਹਰੇ ਦੀ ਅਗਵਾਈ ...
ਹਿਮਾਚਲ ਪ੍ਰਦੇਸ਼ ਤੋਂ ਆਈ ਸ਼ਰਧਾਲੂ ਔਰਤ ਦਾ ਲੁਟੇਰੇ ਪਰਸ ਖੋਹ ਕੇ ਫਰਾਰ
ਅੰਮਿ੍ਤਸਰ, 15 ਅਕਤੂਬਰ (ਰੇਸ਼ਮ ਸਿੰਘ)-ਹਿਮਾਚਲ ਪ੍ਰਦੇਸ਼ ਦੇ ਇਲਾਕੇ ਪਾਉਂਟਾ ਸਾਹਿਬ ਤੋਂ ਆਪਣੇ ਪਰਿਵਾਰ ਸਮੇਤ ਸ੍ਰੀ ਹਰਿਮੰਦਰ ਸਾਹਿਬ ਦੇ ਦਰਸ਼ਨਾ

Data after  cleaning: ੈਟਰੋਲ ਤੇ ਡੀਜ਼ਲ ਦੇ ਭਾਅ ਚ ਕੀਤੇ ਗਏ ਅਥਾਹ ਵਾਧੇ ਅਤੇ ਪੰਜਾਬ ਸਰਕਾਰ ਵਲੋਂ ਚੋਣਾ ਦੌਰਾਨ ਗਰੀਬਾਂ ਤੇ ਮਜਦੂਰਾਂ ਨੂੰ 5 5 
ਅੰਮਿ੍ਤਸਰ, 15 ਅਕਤੂਬਰ ਹਰਮਿੰਦਰ ਸਿੰਘ ਪੰਜਾਬ ਦੇ ਕੈਬਨਿਟ ਮੰਤਰੀ ਨਵਜੋਤ ਸਿੰਘ ਸਿੱਧੂ ਦੇ ਿਖ਼ਲਾਫ਼ ਭਾਜਪਾ ਦੀ ਜ਼ਿਲ੍ਹਾ ਇਕਾਈ ਵਲੋਂ ਸਥਾਨਕ ਹਾਥੀ ਗੇਟ ਦੇ ਬਾਹਰਵਾਰ ਜ਼ੋਰਦਾਰ ਮੁਜ਼ਾਹਰਾ ਕੀਤਾ ਗਿਆ ਤੇ ਉਸ ਦਾ ਪੁਤਲਾ ਸਾੜਿਆ ਗਿਆ ਇਸ ਮੁਜ਼ਾਹਰੇ ਦੀ ਅਗਵਾਈ 
ਹਿਮਾਚਲ ਪ੍ਰਦੇਸ਼ ਤੋਂ ਆਈ ਸ਼ਰਧਾਲੂ ਔਰਤ ਦਾ ਲੁਟੇਰੇ ਪਰਸ ਖੋਹ ਕੇ ਫਰਾਰ
ਅੰਮਿ੍ਤਸਰ, 15 ਅਕਤੂਬਰ ਰੇਸ਼ਮ ਸਿੰਘ ਹਿਮਾਚਲ ਪ੍ਰਦੇਸ਼ ਦੇ ਇਲਾਕੇ ਪਾਉਂਟਾ ਸਾਹਿਬ ਤੋਂ ਆਪਣੇ ਪਰਿਵਾਰ 

In [24]:
# cleaning the data
data = remove_non_punjabi_chars(punj_data)


# Getting the vocabulary of characters
chars = sorted(list(set(data)))
vocab_size = len(chars)
print(f'vocab_size: {vocab_size}')
print(f"unique_charcters: {''.join(chars)}")

# Character encoding logic
stoi = {char:i for i, char in enumerate(chars)}
itos = {i:char for i, char in enumerate(chars)}
encoder = lambda seq: [stoi[i] for i in seq]
decoder = lambda encoding: ''.join([itos[i] for i in encoding])

# Encoding the data
data = torch.tensor(encoder(data), dtype=torch.long)


# Train-test split
train, test = data[:int(0.9*len(data))], data[int(0.9*len(data)):]

vocab_size: 126
unique_charcters: 
 ,0123456789?[।ਁਂਃ਄ਅਆਇਈਉਊ਌਍਎ਏਐਓਔਕਖਗਘਙਚਛਜਝਞਟਠਡਢਣਤਥਦਧਨ਩ਪਫਬਭਮਯਰਲਲ਼਴ਵਸ਼਷ਸਹ਼ਾਿੀੁੂ੃੄੆ੇੈੋੌ੍੎੏ੑ੒੖੗ਖ਼ਗ਼ਜ਼ੜ੝ਫ਼੠੡੢੤੥੦੧੨੩੪੫੬੭੮੯ੰੱੲੳੴੵ੿ંઅઆઇઈઉઋએ‘


In [25]:
# Getting a sample batch from the data split
def get_batch_with_pos(split, batch_size, context_length):
    if split == 'train':
        data = train
    else:
        data = test
        
    #getting random starting indices for the batch_size
    start_indices = torch.randint(
        len(data) - context_length - 1,
        (batch_size,)
    )
    x_y = torch.stack([data[i:i+context_length+1]for i in start_indices], dim=0)
    x, y = x_y[:,:-1], x_y[:,1:]    
    pos = torch.arange(batch_size * context_length).reshape(batch_size, context_length) % context_length
    x, pos, y = x.to(device), pos.to(device), y.to(device)
    return x, pos, y

x, pos, y = get_batch_with_pos('train', 4, context_length)
x, pos, y


(tensor([[ 70,  94,   1,  ...,  61,  23,   1],
         [ 17,   1,  56,  ..., 125,  21,  54],
         [  1,  67,  58,  ...,   1,  58,  73],
         [ 15,   0,  20,  ...,  58,  72,   1]], device='cuda:0'),
 tensor([[  0,   1,   2,  ..., 253, 254, 255],
         [  0,   1,   2,  ..., 253, 254, 255],
         [  0,   1,   2,  ..., 253, 254, 255],
         [  0,   1,   2,  ..., 253, 254, 255]], device='cuda:0'),
 tensor([[94,  1, 38,  ..., 23,  1,  5],
         [ 1, 56, 70,  ..., 21, 54,  1],
         [67, 58, 60,  ..., 58, 73, 33],
         [ 0, 20, 40,  ..., 72,  1, 52]], device='cuda:0'))

In [26]:
class RoPE(nn.Module):
    def __init__(self, base, dim, max_seq_len):
        super(RoPE, self).__init__()
        theta = base ** -(torch.arange(0,dim,2)/dim)
        pos = torch.arange(max_seq_len)
        freq = torch.einsum('i,j->ij', pos, theta)
        self.register_buffer('cos', freq.cos())
        self.register_buffer('sin', freq.sin())
    def forward(self, x):
        B, S, _ = x.shape
        cos = self.cos[:S]
        sin = self.sin[:S]
        a, b = x[:,:,::2], x[:,:,1::2]
        a_cos, b_cos, a_sin, b_sin = a * cos, b * cos, a * sin, b * sin
        # rot(a,b) = a cos(theta) - b sin(theta), a sin(theta) + b cos(theta)
        rot_1, rot_2 = a_cos - b_sin, a_sin + b_cos
        rot = torch.stack((rot_1, rot_2), -1)
        rot_embd = rot.reshape(B, S, -1)
        return rot_embd
        

In [27]:
class FeedFroward(nn.Module):
    def __init__(self, n_embd):
        super(FeedFroward, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd*4),
            nn.ReLU(),
            nn.Linear(n_embd*4, n_embd),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)
    

class AttentionHead(nn.Module):
    def __init__(self, head_dim):
        super(AttentionHead, self).__init__()
        self.head_dim = head_dim
        self.query = nn.Linear(n_embd, self.head_dim) #(B,S,C)
        self.key = nn.Linear(n_embd, self.head_dim) #(B,S,C)
        self.value = nn.Linear(n_embd, self.head_dim) #(B,S,C)
        self.register_buffer('tril', torch.tril(torch.ones(context_length,context_length)))
        self.dropout = nn.Dropout(dropout)
        if rope_embeddings:
            self.rope = RoPE(1e4, head_dim, 2048)

    def forward(self, embed, verbose=False):
        q = self.query(embed)
        k = self.key(embed)
        v = self.value(embed)
        if rope_embeddings:
            q = self.rope(q)
            k = self.rope(k)
        a = q @ k.transpose(-2,-1) * self.head_dim**-0.5
        a = a.masked_fill(self.tril==0, float('-inf'))
        a = F.softmax(a, dim=-1)
        a = self.dropout(a)
        if verbose:
            print(a.shape)
            plt.imshow([[j.item() for j in i]for i in a[0]])

        output = a @ v
        return output
            
class MoEMultiheadAttention(nn.Module):
    def __init__(self, num_heads, head_size, sparsity_factor=0.3):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = head_size
        self.scale = self.head_dim ** -0.5
        self.sparsity_factor = sparsity_factor

        self.qkv_linear = nn.Linear(n_embd, n_embd * 3)
        self.out = nn.Linear(n_embd, n_embd)
        self.gate = nn.Linear(n_embd, n_heads, bias=False)

    def forward(self, x):
        batch_size, seq_length, hidden_size = x.size()

        # Create the keys, queries and values
        qkv = self.qkv_linear(x)
        #print(qkv.shape)
        qkv = qkv.reshape(batch_size, seq_length, self.num_heads, 3 * self.head_dim)
        #print(qkv.shape)
        qkv = qkv.transpose(1, 2)
        #print(qkv.shape)
        queries, keys, values = qkv.chunk(3, dim=-1)
        #print(queries.shape, keys.shape, values.shape)
        # Compute the gate values (scores) for the keys, separate for each head
        gate_scores = self.gate(x)
        gate_scores = gate_scores.transpose(1, 2)

        # Select top-k keys and values based on gate scores
        sparsity_count = int(seq_length * self.sparsity_factor)
        #print('sparsity_count',sparsity_count)
        topk_indices = torch.topk(gate_scores, sparsity_count, dim=-1).indices
        topk_indices = topk_indices.unsqueeze(-1).expand(-1, -1, -1, self.head_dim)
        #print('topk_indices',topk_indices.shape)
        # Gather top-k keys and values
        topk_keys = torch.gather(keys, 2, topk_indices)
        topk_values = torch.gather(values, 2, topk_indices)

        # Compute attention scores and apply softmax
        scores = torch.matmul(queries, topk_keys.transpose(-2, -1)) / self.scale
        attention = F.softmax(scores, dim=-1)
        #print('Attn', attention.shape)

        # Compute context
        #print('topk_values', topk_values.shape)
        context = torch.matmul(attention, topk_values)
        #print('Attn', context.shape)
        context = context.transpose(1, 2).reshape(batch_size, seq_length, hidden_size)

        # Output projection
        output = self.out(context)
        return output


class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, head_size):
        super(MultiHeadAttention, self).__init__()
        self.heads = nn.ModuleList([AttentionHead(head_size) for i in range(n_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, idx, verbose = False):
        output =  torch.cat([head(idx, verbose) for head in self.heads], dim = -1)
        output =  self.proj(output)
        return self.dropout(output)



class Block(nn.Module):
    def __init__(self, n_embd, n_heads):
        super(Block, self).__init__()
        self.mh_attn = MultiHeadAttention(n_heads, n_embd//n_heads)
        #self.mh_attn = MoEMultiheadAttention(n_heads, n_embd//n_heads)
        self.f_frwd = FeedFroward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self,x):
        x = self.ln1(x)
        x = x + self.mh_attn(x)
        x = self.ln2(x)
        x = x + self.f_frwd(x)
        return x
    

class PunjabiAttentionModel(nn.Module):
    def __init__(self):
        super(PunjabiAttentionModel, self).__init__()
        self.token_embedding = nn.Embedding(vocab_size, n_embd)
        if not rope_embeddings:
            self.position_embedding = nn.Embedding(context_length, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_heads) for i in range(n_layers)])
        self.register_buffer('tril', torch.tril(torch.ones(context_length,context_length)))
        self.linear = nn.Linear(n_embd, vocab_size)
        self.norm = nn.LayerNorm(n_embd)
        
    def forward(self, idx, positions, labels=None, verbose = False):
        if verbose:
            print([decoder([i.item() for i in idx[0]])],'\n')
        idx = self.token_embedding(idx)
        #idx = torch.cat((idx,pos_embed), dim=-1)
        if not rope_embeddings:
            pos_embed = self.position_embedding(positions)
            idx += pos_embed
        #idx = self.lm_heads(idx, verbose)
        #logits = self.attention(idx, verbose)
        idx = self.blocks(idx)
        logits = self.linear(idx)
        
        if labels is None:
            loss = None
        else:
            B, S, E = logits.shape
            #print(labels[0], logits[0])
            logits = logits.reshape(B * S, E)
            labels = labels.reshape(B*S)
            loss = F.cross_entropy(logits, labels)
        return logits, loss
        
    def generate(self, idx, pos, max_seq_length, sampling=True):
        for i in range(max_seq_length):
            logits, _ = self(idx[:,-context_length:], pos)
            logits = logits[:, -1, :]
            if sampling:
                probs = F.softmax(logits, -1)
                generated_char_ids = torch.multinomial(probs, 1)
                idx = torch.cat((idx, generated_char_ids),dim=1)
            else:
                generated_char_ids = logits.argmax(-1)
                idx = torch.cat((idx, generated_char_ids.unsqueeze(0).T),dim=1)
        return idx
    

In [28]:
@torch.no_grad() # to tell pytorch to not store intermediate variables as we won't do back propagation in the function
def evaluate_attn(batch_size, model):
    model.eval()
    losses = {}
    for split in ['train', 'eval']:
        x, pos, y = get_batch_with_pos(split, batch_size, context_length)
        _, loss = model(x, pos, y)
        losses[split] = loss.item()
    return losses


model_attn = PunjabiAttentionModel()
model_attn.to(device)
optimizer_attn = torch.optim.AdamW(model_attn.parameters(), lr = learning_rate)



    




In [29]:
for i in tqdm(range(max_iters)):
    if i % eval_interval == 0:
        losses = evaluate_attn(batch_size = eval_iters, model = model_attn)
        print(f'train loss: {losses["train"]}, eval_loss: {losses["eval"]}')
    x, pos, y = get_batch_with_pos('train', batch_size, context_length)
    _, loss = model_attn(x, pos, y)
    optimizer_attn.zero_grad()
    loss.backward()
    optimizer_attn.step()
print(loss.item())   

  0%|          | 0/5000 [00:00<?, ?it/s]

train loss: 4.8726606369018555, eval_loss: 4.867763042449951


 10%|█         | 501/5000 [01:29<25:50,  2.90it/s]

train loss: 1.716081142425537, eval_loss: 1.7225172519683838


 20%|██        | 1001/5000 [02:58<22:58,  2.90it/s]

...

train loss: 1.2839099168777466, eval_loss: 1.3419642448425293


 90%|█████████ | 4501/5000 [13:17<02:52,  2.90it/s]

train loss: 1.284499168395996, eval_loss: 1.304429054260254


100%|██████████| 5000/5000 [14:45<00:00,  5.64it/s]


1.2776238918304443


In [42]:
def generate_text(context, model, gen_len, sampling=True):
    pad = ''.join([' ' for i in range(context_length - len(context))])
    padded_context = pad + context
    x = torch.tensor([encoder(padded_context)], device = device)
    pos = torch.arange(context_length).unsqueeze(0)
    pos = pos.to(device)
    output = model_attn.generate(x,pos, gen_len, sampling)
    print(f'context: {context}\n')
    generation = decoder([i.item() for i in output[0][-gen_len:]])
    print(f'generation: {generation}')
    return generation


context = 'ਪੰਜਾਬ ਸਰਕਾਰ'
gen = generate_text(context, model_attn, 500)

context: ਪੰਜਾਬ ਸਰਕਾਰ

generation:  ਵਲੋਂ ਜਨਵਰੀ 2058 ਪੰਜਾਬੀ ਫੌਜ ਨੇ ਸਿਲੈਰਿੰਗ ਅਤੇ ਕੋਚ ਵਿਚ ਪੰਚਾਇਤ ਸੰਮਤੀ ਨਾਲ ਸਬੰਧਤ ਉਨੇਗਾ ਜਾਰੀ 
ਆਲੋਚਕਾਂ ਦੀ ਸਲਾਹ ਬਿੰਦੂ ? ਦੁਬਈ ਤੋਂ ਮੁਕਤੀ ਬਾਅਦ ਵੀ ਇਹ ਵਾਕ ਬੈਚ ਸ਼ੇਰ ਮਾਮਲੇ ਵਿਚ ਕਰਵਾਇਆ ਸ਼ਿਕਾਇਤਾਂ ਨੂੰ ਛੁਡਾਊ ਲਈ ਯੂਕੇ ਵੱਲੋਂ ਯਾਦ ਕਰਾਇਆ ਗਿਆ ਕਿ ਇੰਟਰਨੈੱਟ ਵੱਲੋਂ ਜਨਰੇਟਰ ਵੀ ਤਿਆਗੇ ਟੂਰਾਂ ਲਈ ਕਹੀ ਗਰਮ ਜਾਗਿੰਗ ਸ਼ਾਮਲ ਸੋਨਾ
ਦੇਸ਼ ਪਿੰਡ ਸ਼ਿਅਨਦ ਵਿਖੇ ਸੈੱਟ ਖੱਬੇ ਦੇ ਵਿਚਾਰਕ ਸ਼ਿਲਾਂਗ ਵਿਚ ਸ਼ਹਿਰ ਦੇ ਹੋਰਨਾਂ ਟੁੱਟੀਆਂ ਹਨ ਤੇ ਬੋਲਦਿਆਂ ਖੁਦਕੁਸ਼ੀਆਂ ਦਾ ਕਹਿਣਾ ਹੈ ਕਿ ਕੀ ਬੋਲੇ ਵਿਖਾਵੇ ਸਿਵਰ ਹੁੰਦੇ ਹਨ, ਜਿਨ੍ਹਾਂ ਸ਼ਿਕਾਇਤਾਂ ਤੇ ਟਿਕਾਣੇ ਦੇ ਬਾਅਦ ਜਿੱਤ ਨਾਲ ਪ੍ਰਾਪਤ ਕਰਨਾ ਚਾਹੁੰਦੇ ਹਨ ਜਦੋਂ ਮੁੜ 


#### producing deterministic output, without sampling

In [47]:
gen = generate_text(context, model_attn, 100, sampling = False)

context: ਪੰਜਾਬ ਸਰਕਾਰ

generation:  ਵੱਲੋਂ ਕੀਤਾ ਗਿਆ ਹੈ ਕਿ ਉਹ ਕਿਸੇ ਵੀ ਕਿਸੇ ਨੂੰ ਕਿਸੇ ਵੀ ਕਿਸੇ ਨੂੰ ਕਿਸੇ ਨਾਲ ਕਿਸੇ ਨਾਲ ਕਿਸੇ ਨੂੰ ਕਿਸੇ ਨਾਲ ਕਿਸੇ 


In [48]:
context = 'ਪੰਜਾਬ ਦੀਆਂ ਚੋਣਾਂ ਜਿੱਤੀਆਂ ਸਨ'
gen = generate_text(context, model_attn, 500)

context: ਪੰਜਾਬ ਦੀਆਂ ਚੋਣਾਂ ਜਿੱਤੀਆਂ ਸਨ

generation:  ਜ਼ਿੱਦ ਕੂੜੀਆਂ ਟਿੱਪਣੀਆਂ ਹੋ ਚੁੱਕੇ, ਉਸਨੇ ਸ਼ੁਰੂ ਕੀਤਾ ਕਿ 
ਪੇਸ਼ਕਦਮੀ ਦਾ ਮੁੱਖ ਮੰਤਰੀ ਹੋ ਕੇ ਤਖ਼ਤ ਹੋ ਕੇ ਸਬੰਧੀ ਸੜਕ ਤੇ ਹੋਏ ਕਿਹਾ ?
ਪਾਣੀ ਬੰਦ 
ਬਈ ਐਸ ਐਸ ਦੀ ਵਰਡ ਕੁਲਵੰਤ ਸਿੰਘ ਨੇ ਸੀਰੀਆ ਤੇ ਮੀਡੀਆ ਐਸੋਸੀਏਸ਼ਨ ‘ਚ ਖੋਜ਼ ਜ ਮੋਦੀ ਭਗਵੰਤ ਮਾਨ ਲਗਾਉਣ ਨਾਲ ਸੋਸਾਇਟੀ ਦੀ ਆਦਤ ਨਹੀਂ ਕਰ ਲਿਆ 
ਰਾਇਸ ਅਕਾਲੀ ਦਲ ਦੇ ਵਿਚਕਾਰ ਐਲਾਨ ਕੀਤਾ ਜ਼ਿੰਦਾ 
ਟ੍ਰੇਨਰਾਂ ਨੂੰ ਟਰੂਡੋ ਕਾਲਾ ਧਨ ਦਾ ਦਾਖ਼ਲਾ ਦਰਜ ਕਰਵਾਇਆ
ਨਵਾਂ ਸਾਲਾਨਾ ਆਮੀਨੀ ਜੀਵਨ ਦੀਆਂ ਨਵੀਂਆਂ ਕਿਸਮਾਂ 21 ਸਤੰਬਰ 1982 ਦੇ ਰਾਖੇ ਚੋਰੀ ਨਸਲਕੁਸ਼ਾ ਸੜਕ ਹਾਦਸੇ ਪੰਜਾਬ ਦਿਵਸ ਵੱਲੋਂ ਤਸਵੀਰ ਕੌਰ ਪੰਜਾਬ ਵਲੋਂ ਖੁਸ਼ੀ ਪਵਾਏ, ਅਜੈਬਲੀ ਅਤੇ ਸੇਦਬੁਰ ਫਰੇਲਰ ਤੋਂ ਓਜਲ ਨਾਲ


In [32]:
path = 'model/punjabi_rope_5k.pth'
torch.save(model_attn.state_dict(), path)

In [35]:
model_loaded = PunjabiAttentionModel()
model_loaded.load_state_dict(torch.load(path))
model_loaded.eval()

PunjabiAttentionModel(
  (token_embedding): Embedding(126, 384)
  (blocks): Sequential(
    (0): Block(
      (mh_attn): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x AttentionHead(
            (query): Linear(in_features=384, out_features=64, bias=True)
            (key): Linear(in_features=384, out_features=64, bias=True)
            (value): Linear(in_features=384, out_features=64, bias=True)
            (dropout): Dropout(p=0.2, inplace=False)
            (rope): RoPE()
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (f_frwd): FeedFroward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
   

In [49]:
gen = generate_text('ਅੱਜ ਦੀ ਖਬਰ', model_attn, 500)

context: ਅੱਜ ਦੀ ਖਬਰ

generation:  ਬਾਦਲ ਅਤੇ ਉਨ੍ਹਾਂ ਨੂੰ ਪੁਨਰਸਨਤਾ ਦੀ ਆਗਿਆ ਤੇ ਉਨ੍ਹਾਂ ਨੂੰ ਬਾਪੂ ਨਾਲ ਗੱਲ ਕਰਦੇ ਹਨ। ਆਪਣੀ ਮੁਟਿਆਰਸ਼ਾਂ ਦੇ ਸੰਤ ਅੱਤਵਾਦੀ ਅੰਦਰ ਪੁਸਤਕ ਦੇ ਮੁਖੀ ਨੂੰ ਤਰੱਕ ਕੇ ਸਮਾਜ ਰਾਹੀ, ਨਸ਼ਿਆਂ ਦੇ ਮੋਹ ਹੈ ਜਿਨ੍ਹਾਂ ਦੇ ਭਿਖੇਰੇ ਇਟਾਲੀਅ ਦੀ ਨਿਵਾਸੀ ਸਮੇਂ ਸਹੂਲਤ ਸਭਨਾ ਨਿਭਾਉਣ ਲਈ ਇਸ ਤੋਂ ਬਾਅਦ ਉਨ੍ਹਾਂ ਤਕ ਅਸਲਰੋਨ ਸਰੀਰ ਵਲੋਂ ਆਤਮਾ ਨਾਲ ਸੂਬਦ ਦੀਆਂ ਛਪੀਆਂ ਖਰੀਆਂ ਕਿਤਾਬ, ਅਤੇ ਕਿਸੇ ਪਾਣੀ ਦੀ ਯਮਾਤਾ ਅਤੇ ਸਬੂਹਾਂ ਤੇ ਬੌਸ ਲਿਖਤ ਪੁਸਤਕਾਂ ਤੱਕ ਹਨ ਜਿਸ ਨੂੰ ਪੈਸੇ ਦਾ ਅਸਾਧਾਰਨ ਕੀਤਾ ਜਾ ਸਕਦੇ ਸਨ ਉਨ੍ਹਾਂ ਦਾ ਸ਼ਿਕਾਰ ਹਵਾਈ ਸ਼ੁਰੂ ਕੀਤਾ ਹੈ, ਆਈਐਸ ਇੰਡੋ , ਜ਼ਬੀਰ ਸਿੰਘ ਖੁਸ਼ੀ ਦੇ ਸਮੇਂ ਅਤੇ ਹਾਇਰਨ ਵੱਲੋਂ ਸਰਕਾਰ ਵਲੋ 


In [50]:
for i in tqdm(range(max_iters)):
    if i % eval_interval == 0:
        losses = evaluate_attn(batch_size = eval_iters, model = model_attn)
        print(f'train loss: {losses["train"]}, eval_loss: {losses["eval"]}')
    x, pos, y = get_batch_with_pos('train', batch_size, context_length)
    _, loss = model_attn(x, pos, y)
    optimizer_attn.zero_grad()
    loss.backward()
    optimizer_attn.step()
print(loss.item())   

  0%|          | 1/5000 [00:00<58:21,  1.43it/s]

train loss: 1.2834171056747437, eval_loss: 1.2880988121032715


 10%|█         | 501/5000 [01:29<25:46,  2.91it/s]

train loss: 1.256445288658142, eval_loss: 1.2932637929916382


 20%|██        | 1001/5000 [02:57<22:58,  2.90it/s]

...

train loss: 1.2133549451828003, eval_loss: 1.192925214767456


 90%|█████████ | 4501/5000 [13:16<02:51,  2.91it/s]

train loss: 1.1670435667037964, eval_loss: 1.187902569770813


100%|██████████| 5000/5000 [14:44<00:00,  5.65it/s]


1.209726095199585


In [51]:
path = 'model/punjabi_rope_10k.pth'
torch.save(model_attn.state_dict(), path)           

In [52]:
gen = generate_text('ਪੰਜਾਬ ਚਾਹੁੰਦਾ ਹੈ', model_attn, 500)

context: ਪੰਜਾਬ ਚਾਹੁੰਦਾ ਹੈ

generation: । ਬਾਓਸਟੇਰੀਆ ਦੀ ਵੱਡੀ ਹਾਈਕਮਾਂਡ ਲਉ ਤੁਹਾਨੂੰ ਹਰ ਹਰਿਆਣਾ ਬਾਰੇ ਖ਼ੁਦ ਕਰਨ ਲਈ ਥਾਂ ਲਿਖਿਆ ਹੈ?
45 ਦਿਨ ਦਾ ਪਰਬੰਧ ਕਰਨਾ ਸ਼੍ਰੋਮਣੀ ਕਮੇਟੀਆਂ ਦੇ ਪਰਿਵਾਰ ਦੀ ਟਹਿਣੀ ਤੇ ਪੈਦਂ ਥੱਲਾ ਸ਼ਹਰਤ ਦੀ ਟਹਿਣੀ ਮਲਿਕਦਾਰ ਹੋਣਾ ਚਾਹੀਦਾ ਹੈ।
ਸਭ ਕੁਝ ਵੀ ਹੈ ਜੋ ਹਿਰਦੈ ਪ੍ਰੋਵੇਕ ਗੁਰਬਾਣੀ ਹੈ। ਇਸ ਲਈ ਜ਼ਿਲ੍ਹਾ ਕਲਫ਼ਿਆਂ ਦੇ ਗਿਅਣ ਨਾ ਜਿਈਏ ਕਹਿਸ਼ ਤੇ ਵਰਦੇ ਵੀ ਛਿਲੈ ਪਰਾਪਤ ਕਰਨ ਨਾਲ ਸਿੱਖਾਂ ਵਾਂਗ ਇਸ ਵੈਬਸ਼ਨ ਅੰਦਰੋਂ ਪੈਦਾ ਕੀਤੇ ਗਏ। ਪਰਿਵਾਰਾਂ ਨੂੰ ਅੱਗੇ ਪਰਵਾਰ ਦੇ ਚੱਕਰ ਦੇ ਨਾਲ ਇੱਟਾਂ ਤੋਂ ਹਟਾ ਕੇ ਖੇਮਾ ਦੇ ਪਰਵਾਨ ਕੀਤੇ ਗਏ। ਜਿਹੜੇ ਬੰਦੇ ਮੇਲਿਆਂ ਵਾਲਿਆਂ ਲਈ 6 ਬਾਲੀਵੁੱਡ ਵਿਚ ਸੈਟੇਲਫਾਈਟ ਸ੍ਟੋਰੀਟਸ ਵਾਲਿਆਂ ਦੇ ਸਭ ਹੁਣ 


In [53]:
gen = generate_text('ਭਾਰਤ ਚਾਹੁੰਦਾ ਹੈ', model_attn, 500)

context: ਭਾਰਤ ਚਾਹੁੰਦਾ ਹੈ

generation: 
ਬਹੁਤ ਵਾਇਰਲ ਹੈ ਜੋ ਸਟਿਮ ਦਿਨ ਡਿਜੀਟਲ ਬਣਾਉਂਦਾ ਹੈ। ਹਰੇਕ ਜੀਪ
ਪੱਤਰਕਾਰ ਨੂੰ ਉਚਿਤ ਪਹਿਲੀ ਹੀ ਆਨੰਦ ਦੇ ਕੇ ਹੱਥ ਹੋਵੇਗਾ ਕਿ ਪਾਰਕੀ ਦੇ ਰੋਹਣੇ ਰੋਂਦੀ ਹੈ ਜਾਂ ਏਡਿਜੀਟਲ ਦੀ ਕਥਿਤ ਦੋਸਤਾਨਾਂ ਦੀ ਚਾਵਲ ਹੈ। 
ਸਟਿੱਕ ਵੀ ਨੋਟ ਕ੍ਰਿਸਟੋਜਨਫਾਰਮੇ ਚ ਰਾਮਗੜ੍ਹ ਚ 19 ਅੱਜ ਤੋਂ
ਇਸਲਾਮਾਬਾਦ ਪਾਕਿਸਤਾਨ ਘਨੌਰ ਨੇ 1989 ਵਿੱਚ ਇੰਦੌਰ ਮਿਊਜ਼ਰ 29 1970 ਇੰਚ ਦੀਆਂ ਅਰਬਾਂ ਖੇਡ ਚਲਦੇ ਰਹਿਣ। ਪਹਿਲੀ ਵਾਰ ਉਹ 92 99 ਇੰਚ 18, 998 
ਓਸਾਮਾ ਬਿਨ ਲਾਦਰੇ
ਅਕਸ਼ੇ ਕਾਮਿਆਂ ਦਾ ਨਾਂ ਖਾਣਾ ਘਟਾਉਣਾ ਚੋਣ ਪ੍ਰਕਿਰਿਆ
1941 ਤੋਂ ਘਟਾਉਣ ਵਾਲੀ ਔਕੜ ਚ ਲੈਕੇ 
1974 ਕਤਲੇਆਮ ਸੜਕ ਤੇ ਦੋਸ਼ੀ ਇਜ਼ਰਾਇਲੀ ਜੋ ਰਾਹੀਂ ਜੀ ਆਇਆਂ ਕੋਲ਼ਾ ਵਿਚ ਅਕਸ਼


In [54]:
gen = generate_text('ਭਾਰਤ ਦੀ ਹਾਕੀ ਟੀਮ', model_attn, 500)

context: ਭਾਰਤ ਦੀ ਹਾਕੀ ਟੀਮ

generation:  ਦਾ ਸੰਤੁਜੇ ਹਾਸਲ ਕਰਨ ਵਾਲਿਆਂ ਤੇ ਬਾਲਾ ਦੀ ਹਾਲਤ ਮਾਰ ਕੇ ਰੱਖੀਂ ਹੀਰੋ ਸਾਹਿਬ ਭਰੇ ਨਵ ਏਜੰਡਾ ਰਾਮ ਕੁਮਾਰ ਦੀ ਥੂਹੀ
 21, 2018ਟਸ। ਪੋਸਟ ਬਿਊਰੋ ਆਪਣੀ ਸੂਚੀ ਦੇ ਨਾਲ ਯੋਗ ਅੱਗੇ ਗੂੜ੍ਹੇ ਦੇ ਰੰਗ ਦੀ ਨਾਵਲ, ਅਣ ਲੋਕ ਗਾਲੀ ਪੁੱਤਰ ਸਾਹਿਬ ਨੇਤ੍ਰਤਾ ਨਾਲ ਸੰਸਥਾ ਦੇ ਮਾਰਵਿਨ ਅਾਵੇ ਭਾਰਤੀ ਮੂਲ ਦਾ ਜਨਮ ਦਿਨ ਲਗਾਇਆ ਸੀ। ਉਨ੍ਹਾਂ ਉਨ੍ਹਾਂ ਦੇ ਇੱਕ ਹਸਪਤਾਲ ਚ ਉਨ੍ਹਾਂ ਦਾ ਯੋਗਦਾਨ 
ਕਰੀਬ ਇੱਕ ਝਲਕ ਦੀ ਇੱਕ ਵਾਰ ਮੈਡਮ ਬੰਦੇ ਦੀ ਪਨੀਤੀ ਘੁਟੋਆ। ਪ੍ਰਧਾਨ ਮੰਤਰੀ
300 ਗੱਲ ਬਾਤ ਯੋਗਦਾਨ ਨਾਮੋ ਬਾਤ ਚਿਨ੍ਹਾਂ ਨੇ ਕਲੱਬ ਲੋਹੇਸ਼ ਦਾ ਮੋਬਾਇਲ ਚਾਰਦੀ ਨਾਲ ਕੀਤਾ। ਬਤੌਰ ਬਾਰਿਸ਼ ਦੀ ਖ਼ੁਰਾਕ ਘਟੀਆ ਮਾਰਚ ਕੀਤੀ ਆਮਦਨ ਸੰਤਾ, ਮਾਹਿਸਾ ਦ


In [55]:
gen = generate_text('ਕ੍ਰਿਕਟ', model_attn, 500)

context: ਕ੍ਰਿਕਟ

generation:  ਸਵਾਰ ਐਥੇਨ ਨੌਮ 857 ਹੋਰ ਮੰਚਲ ਸ਼੍ਰੋਮਣੀ ਗੁਰਦੁਆਰੇ ਦੇ ਸੂਬਾ ਪ੍ਰਧਾਨ ਚੁਣੇ ਗੰਢੀ, ਤਹਿਸੀਲ ਏਅਰ ਏਚੀਟ ਦੇ ਚੇਅਰਮੈਨ, ਕੈਡਕੀ ਵਾਰਸਭਿਅਤਾ ਅਤੇ ਪੂਰਬੀ ਏਸ਼ੀਆ ਚ ਮੌਜੂਦਾ ਪਾਡ਼ਾਸਿਨਰਸਿਪੀ, ਜਥੇਦਾਰ ਅਵਤਾਰ ਸਿੰਘ ਬੱਸੜ ਬਲਵੀਪੁਰ ਦੇ ਲੰੈਕਿਸੇਆਂ ਦੇ ਬਸਤੀ ਚੇਅਰਮੈਨ ਜਗਰਾੳਾਣ ਕਰਦੇ ਹਨ, ਫਿਰ ਉਨ੍ਹਾਂ ਦਿਨੇਸ਼ ਅਤੇ ਸੱਭਿਆਚਾਰਾਂ ਨੂੰ ਨਹੀਂ ਮਿਲਿਆ ਹੈ।
 ਅਕਾਲੀ ਦਲ ਭਾਜਪਾ ਦੇ ਸੂਬਾ ਕੌਂਸਲ ਏਜੰਸੀ ਕੈਪਟਨ ਸਾਰੀਆਂ ਵਿਕੀਪੀਡੀਆ, ਇਕ ਅਜ਼ਾਦੀ ਘਟਨਾ ਅਤੇ ਇਨ੍ਹਾਂ ਨੂੰ ਹੁੰਦਿਆਂ ਚੁਣਿਆ ਗਿਆ ਹੈ। ਕੈਥਲੀਨ ਦੀ ਵਿਦਿਅਕ ਨਈਟਸ ਹਾਂ, ਇਹੀ ਇਸ ਤੋਂ ਪਹਿਲਾਂ ਉਨ੍ਹਾਂ 
ਇੰਨਸਾਫ਼ ਨਾਲ ਸਬੰਧਤ ਮੰਤਰੀ ਨਾਲ ਮੋਹਾਲੀ ਦੀ ਬੇਨਤੀ ਰੀਅਲਜੁਦਾ 


In [59]:
gen = generate_text('ਮੰਤਰੀ', model_attn, 500)

context: ਮੰਤਰੀ

generation:  ਬਣਕੇ ਹੋਣ ਦਾ ਉਪਾਅ ਦਿੱਤਾ।ਹੇਠਾਂ ਦਸੇ ਸਾਰੇ ਵਿਚਾਰੇ ਪੁੱਤਰ ਪਰਮੀਸ਼ ਦੀ ਹੈ ਜੋ ਵਿਆਹ ਬੈਠਣ ਦਾ ਵੀ ਚਾਰ, ਵਾਰਾਂ ਦੀ ਜੀਵਨ ਮਰਜ਼ੀ ਨਾਲ ਆਈ ਕਿਸਾਨੀ ਦਾ ਫ਼ੈਸਲਾ ਲੈਣ ਦਾ ਸਹਾਈ ਜਿਹੜਾ ਕੁਦਰਤ ਬਣੇ ਹੋਏ, ਉਹ ਆਖ਼ਰੀ ਪੁਲੇਸ ਚਾਰੇ ਤੇ ਰਿਸਕਾਰੀ। ‘ਹਾਸਰੀਆਂ ਨੇ ਛੇੜੀ ਜਾ ਉਪਰੇਸ਼ਨ ਕੀਤੀ, ਨੇ ਤੱਕੇ, ਕਿਹਨੂੰ ਧੂਫੀ ਵਿਰੁੱਧ ਅਥਵਾ ਪਾਉਣਾ। ਆਖ਼ਰੀ ਦੀ ਕਮੀ ਹਰੇਕ ਕੋਈ, ਟੱਪ ਰਿਹਾ। ਜੀ ਫੜਨਾ ਲੱਗਦੈ ਹੁੰਦਾ ਹੈ ਅਸੀਂ ਬੇਚੈਨੀ ਭੇਜ ਕੇ ਅੰਦਰਿਆਂ ਚ ਆਉਂਦੇ ਰਹੇ ਝਿੜਕੇ ਤੇ ਸਿਰਫ
ਖ਼ੁਦ, ਇਕ ਤਰਣਾ ਸਵਾਲ ਵੀ ਨਾ ਹੋਈ ।। ਇੱਕ ਦੂਜੇ ਦੇ ਅੰਦਰ ਆਕਾਸ਼ ਉਪਰ ਢੰਗ ਰਿਹਾ ਤੋਂ ਆਵੇ, ਪਹਿਲੇਵਾਂ। ਸ਼ਾਜ ਦਾ ਪਿੰਡ ਰਬੂਬਲ ੱਲਾਣਾ ਚਾਹੁੰਦੀ ਏ।
ਰਾਜਦੇ
